In [ ]:
from utils.cn import Linear
from utils.tensor import Tensor
import numpy as np

m = Linear(n_in=600, n_out=600, bias=True)

x = Tensor(np.random.randn(10, 600))

y = m(x)
y.backward()

print(y)
print(m.parameter.grad)
print(m.bias.grad)

In [85]:
import numpy as np

arr = np.zeros([2,3,4,5,6])
arr = np.array([12])

shape = arr.shape

# a = shape[:2]
# b = shape[2:4]
# c = shape[4:]
# f = a + b + c
# f
shape[:0], shape[0], shape[0+1:]

((), 1, ())

In [39]:
def flatten(arr, start_dim: int = 0, end_dim: int = -1):
    shape = arr.shape

    if start_dim < 0:
        start_dim = len(shape) + start_dim 
    if end_dim < 0:
        end_dim = len(shape) + end_dim + 1
    
    # Compose new shape
    start_shape = shape[:start_dim]
    flatten_dim = np.prod(shape[start_dim:end_dim])
    end_shape = shape[end_dim:]


    final_shape = start_shape + (flatten_dim,) + end_shape

    flat = np.reshape(arr, final_shape)
    return flat

shaped = flatten(arr, 1, -1)
print(shaped.shape)

b = shaped.reshape(shape)
b.shape

(2, 360)


(2, 3, 4, 5, 6)

In [ ]:
import numpy as np
arr = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(arr.size)


In [87]:
5//2

2

In [ ]:
from utils.dataloader import DataLoader, train_dataset, test_dataset
batch_size = 34
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=False)

len(test_loader.dataset)
test_loader.dataset.y

In [ ]:
from utils import candle

candle.is_grad_enabled()

with candle.no_grad():
    print(candle.is_grad_enabled())

In [ ]:
import numpy as np

b = np.array([[1, 2, 3], [4, 5, 6]])
f = np.array([[7, 8], [9, 10]])

b.T @ f

In [45]:
False and True

False

In [43]:
if not False:
    print("False")

False


In [ ]:
(f.T @ b).T

In [ ]:
dict_ = {"side": "test"}

try:
    start_index = dict_["start_index"]
    end_index = dict_["end_index"]
    matrix_side = dict_["matrix"]
except:
    start_index = None
    end_index = None
    matrix_side = None

side = dict_["side"]

if "matrix" in dict_:
    print("test1")

if 2 != None:
    print("test2")

In [ ]:
import numpy as np

# Create two 3D matrices (tensors)
# For valid matrix multiplication with shape (a, b, c) @ (a, c, d) -> (a, b, d)
A = np.random.randint(0, 10, (2, 3, 4))
B = np.random.randint(0, 10, (2, 4, 5))

# Multiply each (i)th 3x4 with (i)th 4x5 -> each result (i) is 3x5. All dims used.
C = np.empty((2, 3, 5), dtype=int)
for i in range(A.shape[0]):
    C[i] = A[i] @ B[i]

print("A.shape:", A.shape)
print("B.shape:", B.shape)
print("C.shape:", C.shape)
print(C)
